In [ ]:
import zipfile
from google.colab import drive
drive.mount('/content/drive')
with zipfile.ZipFile("/content/drive/MyDrive/Colab_Notebooks/data/sii_data/lab_3/miem-hse-ais-2023-lab-03.zip","r") as zip_ref:
    zip_ref.extractall("./")

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
res_df = pd.read_csv("sample_submission.csv")

train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16784 entries, 0 to 16783
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  16784 non-null  int64 
 1   movie_id   16784 non-null  int64 
 2   text       16784 non-null  object
 3   label      16784 non-null  object
dtypes: int64(2), object(2)
memory usage: 524.6+ KB


In [ ]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15231 entries, 0 to 15230
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  15231 non-null  int64 
 1   text       15231 non-null  object
dtypes: int64(1), object(1)
memory usage: 238.1+ KB


In [ ]:
res_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15231 entries, 0 to 15230
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   review_id  15231 non-null  int64
 1   positive   15231 non-null  int64
dtypes: int64(2)
memory usage: 238.1 KB


In [ ]:
res_df.head()

,review_id,positive
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [ ]:
! pip install -U transformers -q

In [ ]:
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
model_name = "cointegrated/rubert-tiny2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=37)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/118M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
X = X_train = train_df['text'].values
y = y_train = train_df['label'].values
labels = np.unique(y)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
encoded_inputs_train = tokenizer(X_train.tolist(), truncation=True, padding=True, max_length=1024)
#encoded_inputs_test = tokenizer(X_test.tolist(), truncation=True, padding=True, max_length=2048)

In [ ]:
input_ids_train = torch.tensor(encoded_inputs_train['input_ids'])
attention_mask_train = torch.tensor(encoded_inputs_train['attention_mask'])
#input_ids_test = torch.tensor(encoded_inputs_test['input_ids'])
#attention_mask_test = torch.tensor(encoded_inputs_test['attention_mask'])
labels_train = torch.tensor([labels.tolist().index(label) for label in y_train])
#labels_test = torch.tensor([labels.tolist().index(label) for label in y_test])

In [ ]:
train_dataset = torch.utils.data.TensorDataset(input_ids_train, attention_mask_train, labels_train)
#test_dataset = torch.utils.data.TensorDataset(input_ids_test, attention_mask_test, labels_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True)
#test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, shuffle=False)

In [ ]:
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
num_epochs = 2

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        batch = tuple(t.to(device) for t in batch)
        input_ids, attention_mask, labels = batch

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1} loss: {total_loss/len(train_loader)}")

In [ ]:
model.eval()
predictions = []
all_labels = []

with torch.no_grad():
    for batch in test_loader:
        batch = tuple(t.to(device) for t in batch)
        input_ids, attention_mask, labels = batch

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions.extend(torch.argmax(logits, dim=1).tolist())
        all_labels.extend(labels.tolist())

In [ ]:
target_names = np.unique(y_test)
print(classification_report(all_labels, predictions, target_names=target_names))

test dataset

In [ ]:
X_test_n = test_df['text'].values

In [ ]:
encoded_inputs_test_n = tokenizer(X_test_n.tolist(), truncation=True, padding=True, max_length=1800)
input_ids_test_n = torch.tensor(encoded_inputs_test_n['input_ids'])
attention_mask_test_n = torch.tensor(encoded_inputs_test_n['attention_mask'])
test_dataset_n = torch.utils.data.TensorDataset(input_ids_test_n, attention_mask_test_n)


test_loader_n = torch.utils.data.DataLoader(test_dataset_n, batch_size=8, shuffle=False)

In [ ]:
#labels_test_n = torch.tensor([labels.tolist().index(label) for label in y_test])

In [ ]:
predictions = []
all_labels = []

with torch.no_grad():
    for batch in test_loader_n:
        batch = tuple(t.to(device) for t in batch)
        input_ids, attention_mask = batch #, labels = batch

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions.extend(torch.argmax(logits, dim=1).tolist())
        #all_labels.extend(labels.tolist())

In [ ]:
# Создайте копию датафрейма test_df
result_df = test_df.copy()

# Удалите столбец 'text'
result_df = result_df.drop('text', axis=1)

# Создайте столбец 'positive' и заполните его значениями из массива predictions
result_df['positive'] = predictions

# Выведите получившийся датафрейм
print(result_df)

In [ ]:
file_name = "Gorbuntsov_BIV_206_lab_3.csv"
result_df.to_csv(file_name, index=False)